In [10]:
#!/usr/bin/env python3
import calendar
import requests
import pandas as pd

# 1) Build your list of periods
#    e.g. ["Mar 2025", "Feb 2025", …, "May 2022"]
periods = []
y, m = 2025, 3
while (y > 2022) or (y == 2022 and m >= 5):
    periods.append(f"{calendar.month_abbr[m]} {y}")
    if m == 1:
        y -= 1
        m = 12
    else:
        m -= 1

all_records = []

# 2) Loop and call the API
#    You’ll need to inspect your browser’s Network tab to find the exact URL
#    and query parameters that return the JSON you see in the page.
API_URL = "https://www.topshelfdata.com/api/listing"  # ← replace with the real endpoint

for period in periods:
    print("Fetching", period)
    # Map your period string into whatever the API expects: maybe YYYY-MM or the literal "Mar 2025"
    payload = {
        "state": "wa",
        "licenseType": "any_license",
        "period": period,    # or e.g. "2025-03" if that’s what you see in DevTools
        # any other required params…
    }
    resp = requests.get(API_URL, params=payload)
    resp.raise_for_status()
    data = resp.json().get("data", [])  # adjust to the actual key
    for item in data:
        item["Period"] = period
        all_records.append(item)

# 3) Dump to CSV
df = pd.DataFrame(all_records)
df.to_csv("washington_businesses_May2022_to_Mar2025.csv", index=False)
print(f"Done! {len(df)} rows saved.")


Selecting period 'Mar 2025' …
  ⚠️ Table not found for 'Mar 2025': Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@role='table']"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6BBDCCF25+75717]
	GetHandleVerifier [0x00007FF6BBDCCF80+75808]
	(No symbol) [0x00007FF6BBB98F9A]
	(No symbol) [0x00007FF6BBBEF4C6]
	(No symbol) [0x00007FF6BBBEF77C]
	(No symbol) [0x00007FF6BBC42577]
	(No symbol) [0x00007FF6BBC173BF]
	(No symbol) [0x00007FF6BBC3F39C]
	(No symbol) [0x00007FF6BBC17153]
	(No symbol) [0x00007FF6BBBE0421]
	(No symbol) [0x00007FF6BBBE11B3]
	GetHandleVerifier [0x00007FF6BC0CD6FD+3223453]
	GetHandleVerifier [0x00007FF6BC0C7CA2+3200322]
	GetHandleVerifier [0x00007FF6BC0E5AD3+3322739]
	GetHandleVerifier [0x00007FF6BBDE69FA+180890]
	GetHandleVerifier [0x00007FF6BBDEE0FF+211

In [11]:
from playwright.sync_api import sync_playwright
import json

def main():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        
        # capture XHR responses
        def on_response(response):
            if response.request.resource_type == "xhr":
                url = response.url
                try:
                    text = response.text()
                except:
                    return
                # look for JSON arrays of objects
                if text.strip().startswith("[") or '"data"' in text:
                    print("----\nXHR URL:", url)
                    try:
                        js = response.json()
                        # print first 2 items if it's a list
                        if isinstance(js, list):
                            print("Sample items:", json.dumps(js[:2], indent=2))
                        # or if it has a data array
                        elif "data" in js and isinstance(js["data"], list):
                            print("Sample items:", json.dumps(js["data"][:2], indent=2))
                    except Exception as e:
                        print("  (could not parse JSON)", e)

        page.on("response", on_response)
        
        # load the page (you can also manipulate the dropdown in the browser to trigger more calls)
        page.goto("https://www.topshelfdata.com/listing/any_license/state/wa?from=2025-03-01&range=")
        
        # give it a few seconds for the React app to fire its XHRs
        page.wait_for_timeout(5000)
        browser.close()

if __name__ == "__main__":
    main()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.